In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

# Data Preparation

## 0 - 30 tourist sites

In [2]:
html_main = requests.get("https://www.tripadvisor.com/Attractions-g32655-Activities-a_allAttractions.true-Los_Angeles_California.html")
bsobj_main = soup(html_main.content, "lxml")

In [11]:
rank = []
tourist_site_name = []
tourist_site_link = []
for s in bsobj_main.find_all("section", class_="_2TabEHya _3YhIe-Un"):
    rank.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[0])
    tourist_site_name.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[1])
    tourist_site_link.append(s.find("div", class_="_3W_31Rvp _1nUIPWja _1l7Rsl_O _3ksqqIVm _2b3s5IMB").a["href"])

## The rest of the tourist sites - 2520 recommended tourist sites in Los Angeles

In [12]:
top_6000 = list(range(30, 6000, 30))

In [13]:
for num in top_6000:
    html_main = requests.get("https://www.tripadvisor.com/Attractions-g32655-Activities-oa" + str(num) + "-Los_Angeles_California.html")
    bsobj_main = soup(html_main.content, "lxml")
    
    for s in bsobj_main.find_all("section", class_="_2TabEHya _3YhIe-Un"):
        rank.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[0])
        tourist_site_name.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[1])
        tourist_site_link.append(s.find("div", class_="_3W_31Rvp _1nUIPWja _1l7Rsl_O _3ksqqIVm _2b3s5IMB").a["href"])

In [14]:
tourist_site_link_update = []
for site in tourist_site_link:
    tourist_site_link_update.append("https://www.tripadvisor.com/" + site)

In [16]:
data = {"Rank": rank, "Tourist Site Name": tourist_site_name, "Site Link": tourist_site_link_update}
df = pd.DataFrame.from_dict(data)
df

,Rank,Tourist Site Name,Site Link
0,1,The Getty Center,https://www.tripadvisor.com//Attraction_Review...
1,2,Griffith Observatory,https://www.tripadvisor.com//Attraction_Review...
2,3,Universal Studios Hollywood,https://www.tripadvisor.com//Attraction_Review...
3,4,Petersen Automotive Museum,https://www.tripadvisor.com//Attraction_Review...
4,5,The Wizarding World of Harry Potter,https://www.tripadvisor.com//Attraction_Review...
...,...,...,...
2515,2516,Sunrise Surf Academy,https://www.tripadvisor.com//Attraction_Review...
2516,2517,Lumber Jack's Axe Throwing,https://www.tripadvisor.com//Attraction_Review...
2517,2518,Chill N Vape Lounge,https://www.tripadvisor.com//Attraction_Review...
2518,2519,Inside Out Restorative Spa,https://www.tripadvisor.com//Attraction_Review...


### 2520 tourist sites in total!

In [18]:
df.to_csv("2520_touristsite.csv")

# Location After Input

In [8]:
df = pd.read_csv("2520_touristsite.csv")
df

,Unnamed: 0,Rank,Tourist Site Name,Site Link
0,0,1,The Getty Center,https://www.tripadvisor.com//Attraction_Review...
1,1,2,Griffith Observatory,https://www.tripadvisor.com//Attraction_Review...
2,2,3,Universal Studios Hollywood,https://www.tripadvisor.com//Attraction_Review...
3,3,4,Petersen Automotive Museum,https://www.tripadvisor.com//Attraction_Review...
4,4,5,The Wizarding World of Harry Potter,https://www.tripadvisor.com//Attraction_Review...
...,...,...,...,...
2513,2513,2514,Sunrise Surf Academy,https://www.tripadvisor.com//Attraction_Review...
2514,2514,2515,In Sheep's Clothing,https://www.tripadvisor.com//Attraction_Review...
2515,2515,2516,Lumber Jack's Axe Throwing,https://www.tripadvisor.com//Attraction_Review...
2516,2516,2517,Universal Tour Group,https://www.tripadvisor.com//Attraction_Review...


In [11]:
df["Tourist Site Name"].str.lower()

0                          the getty center
1                      griffith observatory
2               universal studios hollywood
3                petersen automotive museum
4       the wizarding world of harry potter
                       ...                 
2513                   sunrise surf academy
2514                    in sheep's clothing
2515             lumber jack's axe throwing
2516                   universal tour group
2517                              gold line
Name: Tourist Site Name, Length: 2518, dtype: object

*** may update want_to_go_name to make it consider capital letter/ error message/ give example

In [16]:
def visit_tourist(df, want_to_go_name):
    want_to_go_name = want_to_go_name.lower()
    each_name = want_to_go_name.split(", ")
    
    for name in each_name:
        if name == each_name[0]:
            visit_html = df[["Tourist Site Name", "Site Link"]][df["Tourist Site Name"].str.lower() == name]
        else:
            visit_html = visit_html.append(df[["Tourist Site Name", "Site Link"]][df["Tourist Site Name"].str.lower() == name])
    
    return visit_html

In [17]:
want_to_go_name = str(input("Which tourist sites do you want to go? (*please input the names of the tourist sites separated by a period)"))
visit_html = visit_tourist(df, want_to_go_name)

Which tourist sites do you want to go? (*please input the names of the tourist sites separated by a period)the getty Center, the Broad


In [13]:
def find_location(visit_html):
    
    location = []
    
    for link in visit_html["Site Link"]:
        html = requests.get(link)
        bsobj = soup(html.content, "lxml")
    
        for loc in bsobj.find_all("script", type = "application/ld+json"):
            if "streetAddress" in loc.string:
                find_part_html = loc.string.split("{")
                for street in find_part_html:
                    if "streetAddress" in street:
                        for value in street.split(","):
                            if "streetAddress" in value:
                                street_name = value.split(":")[1][1:][:-1] + ", "
                                   
                            if "addressLocality" in value:
                                city_name = value.split(":")[1][1:][:-1] + ", "
                                 
                            if "postalCode" in value:
                                zipcode = "CA " + value.split(":")[1][1:][:-1]
                            
        location_info = street_name + city_name + zipcode
        location.append(location_info)
    
    return location

In [14]:
# location = find_location(visit_html)

In [22]:
# location

['N Broadway at North Hill St Downtown, Los Angeles, CA 90012',
 '10886 Le Conte Ave, Los Angeles, CA 90024-3098']